In [1]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sy
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


sy.init_printing()

upon_waking = pd.to_datetime("2025-05-01 7am")
pre_breakfast = pd.to_datetime("2025-05-01 8am")
pre_lunch = pd.to_datetime("2025-05-01 12pm")
pre_dinner = pd.to_datetime("2025-05-01 5pm")

%matplotlib inline

/home/accrist/anaconda3/envs/modeling/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [ ]:
I, t, k_ISF, k_abs, k_h, G_h, G, C, k_CHOR = sy.symbols(
    "I t k_ISF, k_abs k_h G_h G C k_CHOR"
)

In [ ]:
k_h * (G_h - G)
-I * k_ISF
+k_CHOR * C
sy.Eq(sy.Derivative(G, t), k_h * (G_h - G) * t - I * k_ISF * t + k_CHOR * C * t)

In [ ]:
sy.Derivative(I * t**2, t)

In [ ]:
sy.diff(I * t**2, t)

In [ ]:
sy.diff(I * t**2, I, evaluate=False)

In [ ]:
# should just do flat x time?

In [ ]:
dt_by_min = pd.date_range(start="2025-05-01", end="2025-05-03", freq="1min")

In [ ]:
ds = pd.DataFrame(index=dt_by_min, data={"insulin_level": np.zeros(len(dt_by_min))})
ds.head()

In [ ]:
ds.info()

In [ ]:
InsulinType = namedtuple("InsulinType", ["agent", "onset", "duration"])
InsulinDose = namedtuple("InsulinDose", ["InsulinType", "dose", "time"])

In [ ]:
glargine = InsulinType(
    "glargine", onset=pd.to_timedelta("2h"), duration=pd.to_timedelta("20h")
)
RAI = InsulinType("RAI", onset=pd.to_timedelta("30min"), duration=pd.to_timedelta("3h"))

In [ ]:
ds = pd.DataFrame(index=dt_by_min, data={"insulin_level": np.zeros(len(dt_by_min))})
ds.head()

In [ ]:
doses = []
for dose in zip(
    [glargine, RAI, RAI, RAI],
    [10, 10, 10, 10],
    [upon_waking, pre_breakfast, pre_lunch, pre_dinner],
):
    doses.append(InsulinDose(InsulinType=dose[0], dose=dose[1], time=dose[2]))
doses

In [ ]:
for dose in doses:
    ds.loc[
        (ds.index >= dose.time + dose.InsulinType.onset)
        & (ds.index < dose.InsulinType.onset + dose.time + dose.InsulinType.duration),
        "insulin_level",
    ] += (
        dose.dose / dose.InsulinType.duration.total_seconds() / 60
    )

In [ ]:
ds.plot()

In [ ]:
ds.resample("30min").mean().plot()

# now what about sigmoidal?